# FileSystem (Polyglot)

In [ ]:
#!import ../nbs/Async.dib

# Async (Polyglot)

# Common (Polyglot)

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

...

Installing Packages FSharp.Control.AsyncSeq..... System.Reactive..... System.Reactive.Linq.....

Installing Packages FSharp.Control.AsyncSeq...... System.Reactive...... System.Reactive.Linq......

Installing Packages FSharp.Control.AsyncSeq....... System.Reactive....... System.Reactive.Linq.......

Installed Packages FSharp.Control.AsyncSeq, 3.2.1 System.Reactive, 5.0.0 System.Reactive.Linq, 6.0.0-preview.1

00010101-0000-0000-0000-0a9876543210
99991231-2359-5999-9999-9a9876543210
19700101-0000-0000-0000-0a9876543210
0001-01-01 00:00:00Z
9999-12-31 23:59:59Z
1970-01-01 00:00:00Z
00000000-0000-0000-00dc-ba9876543210
99999999-9999-9999-99dc-ba9876543210
0
999999999999999999
0
0
20:04:15 #1 [Debug] test
1
1
1
0
20:04:20 #1 [Debug] runWithTimeout / timeout: 10 / exception: The operation has timed out.
<null>
FSharpOption<Unit>
      Value: <null>
20:04:21 #2 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, RemoveHandler ]
20:04:21 #3 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, IObservable.Subscribe, 3, RemoveHandler, IObservable.Dispose ]
20:04:21 #4 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, IObservable.Subscribe, 3, TestEvent.Subscribe(error), TestEvent.Iter(0:error), testEvent.EventInterface.Subscribe(error),

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs

In [ ]:
open Common
open Async

## TempDirectory

In [ ]:
let createTempDirectoryName () =
    let root =
        match System.Reflection.Assembly.GetEntryAssembly().GetName().Name with
        | assemblyName -> assemblyName

    System.IO.Path.GetTempPath ()
    </> $"!{root}"
    </> string (newGuidFromDateTime System.DateTime.Now)

let createTempDirectory () =
    let tempFolder = createTempDirectoryName ()
    let result = System.IO.Directory.CreateDirectory tempFolder

    let getLocals () =
        $"tempFolder: {tempFolder} / result: {({|
            Exists = result.Exists
            CreationTime = result.CreationTime
        |})} {getLocals ()}"

    trace Debug (fun () -> "createTempDirectory") getLocals

    tempFolder

In [ ]:
//// test

createTempDirectoryName ()

C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2172-7283-7469b78c33f7

## FileSystemWatcher

In [ ]:
[<RequireQualifiedAccess>]
type FileSystemChangeType =
    | Error
    | Changed
    | Created
    | Deleted
    | Renamed

[<RequireQualifiedAccess>]
type FileSystemChange =
    | Error of exn: exn
    | Changed of path: string * content: string option
    | Created of path: string * content: string option
    | Deleted of path: string
    | Renamed of oldPath: string * (string * string option)


let watchWithFilter filter shouldReadContent path =
    let fullPath = System.IO.Path.GetFullPath path
    let getLocals () = $"fullPath: {fullPath} / filter: {filter} / {getLocals ()}"

    let watcher =
        new System.IO.FileSystemWatcher (
            Path = fullPath,
            NotifyFilter = filter,
            EnableRaisingEvents = true,
            IncludeSubdirectories = true
        )

    let getEventPath (path : string) = path.Trim().Replace(fullPath, "").TrimStart [| '/'; '\\' |]

    let ticks () = System.DateTime.UtcNow.Ticks

    let readContent fullPath =
        if not shouldReadContent
        then None
        else
            try
                System.Threading.Thread.Sleep 1
                System.IO.File.ReadAllText fullPath |> Some
            with ex ->
                trace Error (fun () -> $"Failed to read file content: {ex.Message}") getLocals
                None

    let changedStream =
        AsyncSeq.subscribeEvent
            watcher.Changed
            (fun event ->
                ticks (),
                [ FileSystemChange.Changed (getEventPath event.FullPath, readContent event.FullPath) ]
            )

    let deletedStream =
        AsyncSeq.subscribeEvent
            watcher.Deleted
            (fun event ->
                ticks (),
                [ FileSystemChange.Deleted (getEventPath event.FullPath) ]
            )

    let createdStream =
        AsyncSeq.subscribeEvent
            watcher.Created
            (fun event ->
                let path = getEventPath event.FullPath
                let content = readContent event.FullPath
                ticks (), [
                    FileSystemChange.Created (path, content)
                    if OS.isWindows () then
                        FileSystemChange.Changed (path, content)
                ])

    let renamedStream =
        AsyncSeq.subscribeEvent
            watcher.Renamed
            (fun event ->
                ticks (), [
                    FileSystemChange.Renamed (
                        getEventPath event.OldFullPath,
                        (getEventPath event.FullPath, readContent event.FullPath)
                    )
                ]
            )

    let errorStream =
        AsyncSeq.subscribeEvent
            watcher.Error
            (fun event -> ticks (), [ FileSystemChange.Error (event.GetException ()) ])

    let stream =
        [
            changedStream
            deletedStream
            createdStream
            renamedStream
            errorStream
        ]
        |> FSharp.Control.AsyncSeq.mergeAll
        |> FSharp.Control.AsyncSeq.map (fun (n, events) ->
            events
            |> List.fold
                (fun (i, events) event ->
                    i + 1L,
                    (n + i, event) :: events)
                (0L, [])
            |> snd
            |> List.rev
        )
        |> FSharp.Control.AsyncSeq.concatSeq

    let disposable =
        Object.newDisposable (fun () ->
            trace Debug (fun () -> "Disposing watch stream") getLocals
            watcher.EnableRaisingEvents <- false
            watcher.Dispose ()
        )

    stream, disposable

let watch path =
    watchWithFilter
        (System.IO.NotifyFilters.Attributes
        ||| System.IO.NotifyFilters.CreationTime
        ||| System.IO.NotifyFilters.DirectoryName
        ||| System.IO.NotifyFilters.FileName
        //  ||| System.IO.NotifyFilters.LastAccess
        //  ||| System.IO.NotifyFilters.LastWrite
        ||| System.IO.NotifyFilters.Security
        //  ||| System.IO.NotifyFilters.Size
        )
        path

### testEventsRaw (test)

In [ ]:
//// test

let testEventsRaw (watchFn : bool -> string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable) write =
    let tempDirectory = createTempDirectory ()
    let stream, disposable = watchFn true tempDirectory

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsync (fun event -> async { events.Add event })

    let run = async {
        let! child = iter () |> Async.StartChild
        do! Async.Sleep 150
        do! write tempDirectory |> Async.AwaitTask
        do! child
    }

    try
        run
        |> Async.runWithTimeout 500
        |> ignore
    finally
        disposable.Dispose ()
        System.IO.Directory.Delete (tempDirectory, true)

    let eventsLog =
        events
        |> Seq.toList
        |> List.sortBy fst
        |> List.fold
            (fun (prev, acc) (ticks, event) -> ticks, (ticks, (if prev = 0L then 0L else ticks - prev), event) :: acc)
            (0L, [])
        |> snd
        |> List.rev
        |> List.map (fun (diff, n, event) -> $"{n} / {diff} / {event}")
        |> String.concat "\n"
    let getLocals () = $"eventsLog: \n{eventsLog} / {getLocals ()}"
    trace Debug (fun () -> "testEventsRaw") getLocals

    events
    |> Seq.toList
    |> List.sortBy fst
    |> List.map snd
    |> List.fold
        (fun (map, acc) event ->
            match event with
            | FileSystemChange.Created (path, content) ->
                (map |> Map.add path None), event :: acc
            | FileSystemChange.Changed (path, Some content) ->
                if map.ContainsKey path && map.[path] = Some content
                then map, acc
                else (map |> Map.add path (Some content)), event :: acc
            | FileSystemChange.Deleted path ->
                (map |> Map.remove path), event :: acc
            | FileSystemChange.Renamed (oldPath, (newPath, content)) ->
                (map |> Map.remove oldPath |> Map.remove newPath), event :: acc
            | _ -> map, event :: acc
        )
        (Map.empty, [])
    |> snd
    |> List.rev

In [ ]:
//// test

let write path = task {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"a{i}")

    do! Async.Sleep 30

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"b{i}")

    do! Async.Sleep 30

    for i = 1 to n do
        System.IO.File.Move (path </> $"file{i}.txt", path </> $"file_{i}.txt")

    do! Async.Sleep 30

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"c{i}")

    do! Async.Sleep 30

    for i = 1 to n do
        System.IO.File.Delete (path </> $"file_{i}.txt")
}

let events = testEventsRaw watch write

events
|> _sequenceEqual [
    FileSystemChange.Created ("file1.txt", Some "a1")
    FileSystemChange.Changed ("file1.txt", Some "a1")
    FileSystemChange.Created ("file2.txt", Some "a2")
    FileSystemChange.Changed ("file2.txt", Some "a2")

    FileSystemChange.Changed ("file1.txt", Some "b1")
    FileSystemChange.Changed ("file2.txt", Some "b2")

    FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "b1"))
    FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "b2"))

    FileSystemChange.Changed ("file_1.txt", Some "c1")
    FileSystemChange.Changed ("file_2.txt", Some "c2")

    FileSystemChange.Deleted "file_1.txt"
    FileSystemChange.Deleted "file_2.txt"
]

20:04:22 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2272-7207-7f8418359782 / result: { CreationTime = 2023-07-18 8:04:22 PM
  Exists = true }
20:04:23 #2 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
20:04:23 #3 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2272-7207-7f8418359782 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
20:04:23 #4 [Debug] testEventsRaw / eventsLog: 
0 / 638253182629419500 / Created ("file1.txt", Some "a1")
1 / 638253182629419501 / Changed ("file1.txt", Some "a1")
155798 / 638253182629575299 / Created ("file2.txt", Some "a2")
1 / 638253182629575300 / Changed ("file2.txt", Some "a2")
171578 / 638253182629746878 / Changed ("file1.txt", Some "b1")
146828 / 638253182629893706 / Changed ("file2.txt", Some "b2")
326564 / 638253182630220270 / Renamed ("file1.txt", ("file_1.txt", Some "b1"))
1393

### testEventsSorted (test)

In [ ]:
//// test

let sortEvent event =
    match event with
    | FileSystemChange.Error _ -> 0
    | FileSystemChange.Created _ -> 1
    | FileSystemChange.Changed _ -> 2
    | FileSystemChange.Renamed (_oldPath, _) -> 3
    | FileSystemChange.Deleted _ -> 4

let formatEvents events =
    events
    |> Seq.toList
    |> List.sortBy (snd >> sortEvent)
    |> List.choose (fun (ticks, event) ->
        match event with
        | FileSystemChange.Error _ ->
            None
        | FileSystemChange.Changed (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Changed)
        | FileSystemChange.Created (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Created)
        | FileSystemChange.Deleted path ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Deleted)
        | FileSystemChange.Renamed (_oldPath, (path, _)) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Renamed)
    )
    |> List.sortBy (fun (_, path, _) -> path)
    |> List.distinctBy (fun (_, path, event) -> path, event)

let testEventsSorted (watchFn : string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable) write =
    let path = createTempDirectory ()
    let stream, disposable = watchFn path

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsync (fun event -> async { events.Add event })

    let run = async {
        let! child = iter () |> Async.StartChild
        do! Async.Sleep 150
        do! write path |> Async.AwaitTask
        do! child
    }

    try
        run
        |> Async.runWithTimeout 500
        |> ignore
    finally
        disposable.Dispose ()
        System.IO.Directory.Delete (path, true)

    let events = formatEvents events

    let eventMap =
        events
        |> List.map (fun (ticks, path, event) -> path, (event, ticks))
        |> List.groupBy fst
        |> List.map (fun (path, events) ->
            let event, _ticks =
                events
                |> List.map snd
                |> List.sortByDescending snd
                |> List.head

            path, event
        )
        |> Map.ofList

    let eventList =
        events
        |> List.map (fun (_ticks, path, event) -> path, event)

    eventMap, eventList

#### create and delete (test)

In [ ]:
//// test

let write path = task {
    let n = 3

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}")

    for i = 1 to n do
        System.IO.File.Delete (path </> $"file{i}.txt")
}

let eventMap, eventList = testEventsSorted (watch false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file1.txt", nameof FileSystemChangeType.Deleted

    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Deleted

    "file3.txt", nameof FileSystemChangeType.Created
    "file3.txt", nameof FileSystemChangeType.Changed
    "file3.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Deleted
    "file2.txt", nameof FileSystemChangeType.Deleted
    "file3.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

20:04:23 #5 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2367-6750-6c2978133069 / result: { CreationTime = 2023-07-18 8:04:23 PM
  Exists = true }
20:04:24 #6 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
20:04:24 #7 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2367-6750-6c2978133069 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
( file3.txt, Created )
( file3.txt, Changed )
( file3.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted
      - Key: file3.txt
        Value: Deleted


#### change (test)

In [ ]:
//// test

let write path = task {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}")

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "")

    for i = 1 to n do
        System.IO.File.Delete (path </> $"file{i}.txt")
}

let eventMap, eventList = testEventsSorted (watch false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file1.txt", nameof FileSystemChangeType.Deleted

    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Deleted
    "file2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

20:04:24 #8 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2432-3236-3a96c6995178 / result: { CreationTime = 2023-07-18 8:04:24 PM
  Exists = true }
20:04:24 #9 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
20:04:24 #10 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2432-3236-3a96c6995178 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted


#### rename (test)

In [ ]:
//// test

let write path = task {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}")

    for i = 1 to n do
        System.IO.File.Move (path </> $"file{i}.txt", path </> $"file_{i}.txt")

    for i = 1 to n do
        System.IO.File.Delete (path </> $"file_{i}.txt")
}

let eventMap, eventList = testEventsSorted (watch false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed

    "file_1.txt", nameof FileSystemChangeType.Renamed
    "file_1.txt", nameof FileSystemChangeType.Deleted

    "file_2.txt", nameof FileSystemChangeType.Renamed
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Deleted
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

20:04:24 #11 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2494-9461-900783c89ea9 / result: { CreationTime = 2023-07-18 8:04:24 PM
  Exists = true }
20:04:25 #12 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
20:04:25 #13 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2494-9461-900783c89ea9 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted


#### full (test)

In [ ]:
//// test

let write path = task {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}")

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "")

    for i = 1 to n do
        System.IO.File.Move (path </> $"file{i}.txt", path </> $"file_{i}.txt")

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"{i}")

    for i = 1 to n do
        System.IO.File.Delete (path </> $"file_{i}.txt")
}

let eventMap, eventList = testEventsSorted (watch false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed

    "file_1.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Renamed
    "file_1.txt", nameof FileSystemChangeType.Deleted

    "file_2.txt", nameof FileSystemChangeType.Changed
    "file_2.txt", nameof FileSystemChangeType.Renamed
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Deleted
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

20:04:25 #14 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2562-6286-69dfdfad2367 / result: { CreationTime = 2023-07-18 8:04:25 PM
  Exists = true }
20:04:26 #15 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
20:04:26 #16 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230718-2004-2562-6286-69dfdfad2367 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Changed )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted
